In [1]:
import torch
from torch import nn
from torch.utils import data
from tqdm import tqdm
from utils.fashion_mnist import load_data_fashion_mnist

In [2]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [3]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


In [2]:
batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size=batch_size)

In [3]:
def evaluate(model:torch.Tensor, dataloader:data.DataLoader, device:str) -> float:
    model.to(device)
    model.eval()

    total = 0
    correct = 0
    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)
        y_hat = model(X)
        
        predict = torch.argmax(y_hat, dim=1)
        correct += (predict == y).sum().item()
        total += y.shape[0]

    model.train()
    
    return correct / total

In [4]:
def train(model:torch.Tensor, dataloader:data.DataLoader, optim:torch.optim.Optimizer, criterion:torch.nn, test_data:data.DataLoader, num_epochs:int=10, device:str="cuda") -> None:
    model.to(device)
    model.train()

    for _ in range(num_epochs):
        total = 0
        total_loss = 0.0
        total_correct = 0.0

        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))

        for i, (X, y) in progress_bar:
            X, y = X.to(device), y.to(device)

            optim.zero_grad()
            y_hat = model(X)
            l = criterion(y_hat, y)
            l.backward()
            optim.step()

            total_loss += l.item()
            predict = torch.argmax(y_hat, dim=1)
            total += y.shape[0]
            total_correct += (predict.type(y.dtype) == y).sum().item()

            progress_bar.set_description(f"Epoch {_}")
            progress_bar.set_postfix(loss=total_loss/(i+1), accuracy=100.*total_correct/total, Learning_rate=optim.param_groups[0]['lr'])

        test_acc = evaluate(model, test_data, device)
        accuracy = 100 * total_correct / total
        print(f"Loss: {total_loss / len(dataloader)}, Accuracy: {accuracy}%, test_acc: {test_acc}")

    

In [7]:
optim = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [8]:
train(net, train_iter, optim, criterion, test_iter, num_epochs=20)

100%|██████████| 157/157 [00:00<00:00, 241.52it/s]

Loss: 1.1817171348691748, Accuracy: 55.27166666666667%, test_acc: 0.7019



100%|██████████| 157/157 [00:00<00:00, 220.35it/s]

Loss: 0.6748720108827294, Accuracy: 73.81333333333333%, test_acc: 0.7398



100%|██████████| 157/157 [00:00<00:00, 227.62it/s]

...

Loss: 0.3073392922181819, Accuracy: 88.50166666666667%, test_acc: 0.8621



100%|██████████| 157/157 [00:00<00:00, 258.79it/s]

Loss: 0.30100927258859567, Accuracy: 88.79166666666667%, test_acc: 0.8735



100%|██████████| 157/157 [00:00<00:00, 316.07it/s]

Loss: 0.2950122020423794, Accuracy: 89.085%, test_acc: 0.8743


In [24]:
class ImproveLeNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2), nn.Tanh(), # [28, 28]
            nn.Dropout(0.2),
            nn.Conv2d(32, 48, kernel_size=3, padding=1), nn.Tanh(), # [28, 28]
            nn.MaxPool2d(kernel_size=2, stride=2), # [14, 14]
            nn.Conv2d(48, 16, kernel_size=5), nn.Tanh(), # [10, 10]
            nn.MaxPool2d(kernel_size=2, stride=2),  # [5, 5]
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 128), nn.LeakyReLU(),
            nn.Linear(128, 64), nn.LeakyReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, X):
        return self.net(X)

In [25]:
improveModel = ImproveLeNet()
optim = torch.optim.Adam(improveModel.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
train(improveModel, train_iter, optim, criterion, test_iter, num_epochs=20)

100%|██████████| 40/40 [00:00<00:00, 98.35it/s]

Loss: 0.657974954615248, Accuracy: 76.14666666666666%, test_acc: 0.8429



100%|██████████| 40/40 [00:00<00:00, 88.78it/s]

Loss: 0.3712228908817819, Accuracy: 86.22666666666667%, test_acc: 0.8663



100%|██████████| 40/40 [00:00<00:00, 94.06it/s]

...

Loss: 0.14292468247895546, Accuracy: 94.6%, test_acc: 0.9127



100%|██████████| 40/40 [00:00<00:00, 102.44it/s]

Loss: 0.13594677419738566, Accuracy: 95.00166666666667%, test_acc: 0.9087



100%|██████████| 40/40 [00:00<00:00, 93.41it/s]

Loss: 0.13424210963731117, Accuracy: 94.95%, test_acc: 0.9149
